In [1]:
import gym
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import matplotlib.pyplot as plt

def one_hot(x):
    return np.identity(48)[x:x+1, :]

env = gym.make("CliffWalking-v0", render_mode="human")

input_size = env.observation_space.n
output_size = env.action_space.n
learning_rate = 0.1

X = tf.placeholder(shape=[1, input_size], dtype=tf.float32)
W = tf.Variable(tf.random_uniform([input_size, output_size], 0, 0.01))
Qpred = tf.matmul(X, W)
Y = tf.placeholder(shape=[1, output_size], dtype=tf.float32)

loss = tf.reduce_sum(tf.square(Y - Qpred))
train = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

disco = 0.95
num_episodes = 10000
rList = []

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for i in range(num_episodes):
        s = env.reset()[0]  
        e = 1. / ((i / 50) + 10)
        rAll = 0
        done = False

        while not done:
            Qs = sess.run(Qpred, feed_dict={X: one_hot(s)})
            if np.random.rand(1) < e:
                a = env.action_space.sample()
                
            else:
                a = np.argmax(Qs)

            result = env.step(a)
            s1=result[0]
            reward=result[1]
            done=result[2]
            _=result[3]
            
            if done:
                reward -= 0.1
                Qs[0, a] = reward
                
            
            else:
                Qs1 = sess.run(Qpred, feed_dict={X: one_hot(s1)})
                Qs[0, a] = reward + disco * np.max(Qs1)

            sess.run(train, feed_dict={X: one_hot(s), Y: Qs})

            rAll += reward
            
            s = s1
            print(reward)
            
                  
        
        rList.append(rAll)
env.close()

print("percentage of successful episodes: " + str((sum(rList) / num_episodes) * 100) + "%")

plt.bar(range(len(rList)),rList)
plt.show()

Instructions for updating:
non-resource variables are not supported in the long term


ValueError: Cannot feed value of shape (0, 16) for Tensor Placeholder:0, which has shape (1, 48)

: 